In [1]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 57.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [3]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [6]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(5)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
%%time
def model_als(factors,iterations, regularization, sparse_user_item):
  model = AlternatingLeastSquares(factors=factors,
                                regularization=regularization,
                                iterations=iterations,
                                calculate_training_loss=True,
                                num_threads=4, random_state=42)

  model.fit(sparse_user_item,
          show_progress=True)

  return model


CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.78 µs


In [9]:
def get_recommendations(user, model, sparse_user_item, N=5):
    res = [id_to_itemid[rec] for rec in
                    model.recommend(userid=userid_to_id[user],
                                    user_items=sparse_user_item[userid_to_id[user]],
                                    N=N,
                                    filter_already_liked_items=False,
                                    filter_items=[itemid_to_id[999999]],
                                    recalculate_user=True)[0]]
    return res

In [10]:
def precision_at_k(recommended_list, actual_list, k=5):
    actual_list = np.array(actual_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(recommended_list, actual_list)
    precision = flags.sum() / k

    return precision


###рассчитаем precision_at_k для разных вариантов параметров модели

In [11]:
for f in [50, 100, 150]:
  for iter in [5, 10, 15]:
    model = model_als(f,iter, regularization = 0.001,sparse_user_item=sparse_user_item)
    result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, sparse_user_item=sparse_user_item, N=5))
    res_precision = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
    print(f'при factors = {f} и iterations = {iter} получаем precision_at_k = {res_precision}')

  0%|          | 0/5 [00:00<?, ?it/s]

при factors = 50 и iterations = 5 получаем precision_at_k = 0.19049951028403525


  0%|          | 0/10 [00:00<?, ?it/s]

при factors = 50 и iterations = 10 получаем precision_at_k = 0.18805093046033303


  0%|          | 0/15 [00:00<?, ?it/s]

...

при factors = 150 и iterations = 5 получаем precision_at_k = 0.17639569049951032


  0%|          | 0/10 [00:00<?, ?it/s]

при factors = 150 и iterations = 10 получаем precision_at_k = 0.16395690499510285


  0%|          | 0/15 [00:00<?, ?it/s]

при factors = 150 и iterations = 15 получаем precision_at_k = 0.1615083251714006


###из данных вариантов лучший результат показала модель при factors = 50 и iterations = 5

###попробуем применить взвешивание к матрице и снова просчитать precision_at_k с разными параметрами

In [12]:
tfidf_user_matrix = tfidf_weight(user_item_matrix.T).T
sparse_tfidf_user_matrix = csr_matrix(tfidf_user_matrix).tocsr()

In [13]:
for f in [50, 100, 150]:
  for iter in [5, 10, 15]:
    model = model_als(f,iter, regularization = 0.001, sparse_user_item=sparse_tfidf_user_matrix)
    result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, sparse_user_item=sparse_user_item, N=5))
    res_precision = result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()
    print(f'при factors = {f} и iterations = {iter} получаем precision_at_k = {res_precision}')

  0%|          | 0/5 [00:00<?, ?it/s]

при factors = 50 и iterations = 5 получаем precision_at_k = 0.1981390793339863


  0%|          | 0/10 [00:00<?, ?it/s]

при factors = 50 и iterations = 10 получаем precision_at_k = 0.19461312438785505


  0%|          | 0/15 [00:00<?, ?it/s]

...

при factors = 150 и iterations = 5 получаем precision_at_k = 0.18844270323212536


  0%|          | 0/10 [00:00<?, ?it/s]

при factors = 150 и iterations = 10 получаем precision_at_k = 0.1823702252693438


  0%|          | 0/15 [00:00<?, ?it/s]

при factors = 150 и iterations = 15 получаем precision_at_k = 0.18041136141038197


###применение взвешивания к матрице позволило улучшить метрику, при этом по прежнему лучший результат при параметрах factors = 50 и iterations = 5

###попробуем применить ВМ25 взвешивание

In [14]:
#bm25_user_item = bm25_weight(user_item_matrix).tocsr()
bm25_user_item = bm25_weight(user_item_matrix.T).T
sparse_bm25_user_item = csr_matrix(bm25_user_item).tocsr()

In [15]:
for f in [50, 100, 150]:
  for iter in [5, 10, 15]:
    model = model_als(f,iter, regularization = 0.001, sparse_user_item=sparse_bm25_user_item)
    result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, sparse_user_item=sparse_user_item, N=5))
    res_precision = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
    print(f'при factors = {f} и iterations = {iter} получаем precision_at_k = {res_precision}')

  0%|          | 0/5 [00:00<?, ?it/s]

при factors = 50 и iterations = 5 получаем precision_at_k = 0.20146914789422138


  0%|          | 0/10 [00:00<?, ?it/s]

при factors = 50 и iterations = 10 получаем precision_at_k = 0.2031341821743389


  0%|          | 0/15 [00:00<?, ?it/s]

...

при factors = 150 и iterations = 5 получаем precision_at_k = 0.23663075416258572


  0%|          | 0/10 [00:00<?, ?it/s]

при factors = 150 и iterations = 10 получаем precision_at_k = 0.23251714005876595


  0%|          | 0/15 [00:00<?, ?it/s]

при factors = 150 и iterations = 15 получаем precision_at_k = 0.22948090107737515


###использование bm2-взвешивания позволило значительно улучшить метрику, наилучший результат получили при factors = 150 и iterations = 5